# [CRQ1]: Does the fare for mile change across NY's borough? We want to discover whether the expenses of a user that enjoys Taxis in one zone is different from those that uses it in another one.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

In [2]:
#Month of JANUARY:
#We imported only the columns useful to answer our Rquestion and data for which trip_distance != 0.

data=pd.read_csv('/Users/Enzopc/Desktop/yellow_tripdata_2018-01.csv', usecols=['tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID', 'DOLocationID', 'fare_amount'])
zone=pd.read_csv('/Users/Enzopc/Desktop/taxi _zone_lookup.csv', usecols=['LocationID', 'Borough'])
datazone=pd.merge(data, zone, how='left', left_on=['PULocationID'],right_on=['LocationID'])
datazone=datazone[datazone.trip_distance != 0]

print("Number of rows:", datazone.shape[0])
print("Number of columns: ", datazone.shape[1])
datazone.head()

Number of rows: 8704498
Number of columns:  8


,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,fare_amount,LocationID,Borough
0,2018-01-01 00:21:05,2018-01-01 00:24:23,0.5,41,24,4.5,41,Manhattan
1,2018-01-01 00:44:55,2018-01-01 01:03:05,2.7,239,140,14.0,239,Manhattan
2,2018-01-01 00:08:26,2018-01-01 00:14:21,0.8,262,141,6.0,262,Manhattan
3,2018-01-01 00:20:22,2018-01-01 00:52:51,10.2,140,257,33.5,140,Manhattan
4,2018-01-01 00:09:18,2018-01-01 00:27:06,2.5,246,239,12.5,246,Manhattan


In [3]:
#transformation from string to datetime
datazone['tpep_pickup_datetime']=pd.to_datetime(datazone['tpep_pickup_datetime'], format= "%Y-%m-%d %H:%M:%S")
datazone['tpep_dropoff_datetime']=pd.to_datetime(datazone['tpep_dropoff_datetime'],format= "%Y-%m-%d %H:%M:%S")

In [4]:
#trip_time: difference between dropoff and pickup
trip_duration= datazone['tpep_dropoff_datetime'] - datazone['tpep_pickup_datetime']

In [5]:
#insert the new column
datazone.insert(loc=2, column='trip_duration', value= trip_duration)

In [6]:
#Conversion in minutes
datazone.loc[:,'minutes_trip_duration']= datazone['trip_duration'].dt.seconds/60
datazone['minutes_trip_duration']= round(datazone['minutes_trip_duration'].astype('float64'),2)

In [7]:
datazone['fare_amount'].describe()

count    8.704498e+06
mean     1.218203e+01
std      1.108726e+01
min     -4.200000e+02
25%      6.500000e+00
50%      9.000000e+00
75%      1.350000e+01
max      8.016000e+03
Name: fare_amount, dtype: float64

***Data Cleaning***
Fare_amount: some values are negative. The cases were considered as being machine errors during the data entry. As done for RQ4, they were replaced by their absolute values. Furthermore, as the minimum fare_amount that is chargeable for any service is $2.5, every transaction falling below that amount was replaced by the median value of the fare_amount 9.0 (50th percentile)

In [8]:
def Clean_data(data):
    data.fare_amount = data.fare_amount.abs()
    indices_oi = data[(data.fare_amount<2.5)].index
    data.loc[indices_oi,'fare_amount'] = 9.0

In [9]:
Clean_data(datazone)

In [10]:
datazone['fare_amount'].describe()

count    8.704498e+06
mean     1.218917e+01
std      1.108002e+01
min      2.500000e+00
25%      6.500000e+00
50%      9.000000e+00
75%      1.350000e+01
max      8.016000e+03
Name: fare_amount, dtype: float64

In [11]:
#Considering the fare amount:  Compute the price per mile equation for each trip.
priceXmile= datazone['fare_amount']/datazone['trip_distance']

In [12]:
#insert the new column
datazone.insert(loc=2, column='priceXmile', value= priceXmile)
datazone.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,priceXmile,trip_duration,trip_distance,PULocationID,DOLocationID,fare_amount,LocationID,Borough,minutes_trip_duration
0,2018-01-01 00:21:05,2018-01-01 00:24:23,9.000000,00:03:18,0.5,41,24,4.5,41,Manhattan,3.30
1,2018-01-01 00:44:55,2018-01-01 01:03:05,5.185185,00:18:10,2.7,239,140,14.0,239,Manhattan,18.17
2,2018-01-01 00:08:26,2018-01-01 00:14:21,7.500000,00:05:55,0.8,262,141,6.0,262,Manhattan,5.92
3,2018-01-01 00:20:22,2018-01-01 00:52:51,3.284314,00:32:29,10.2,140,257,33.5,140,Manhattan,32.48
4,2018-01-01 00:09:18,2018-01-01 00:27:06,5.000000,00:17:48,2.5,246,239,12.5,246,Manhattan,17.80


In [15]:
#Run the mean of the new variable for each borough
means=datazone.groupby('Borough').agg(np.mean)['priceXmile']

In [16]:
#Run the standard deviation of the new variable for each borough
sds=datazone.groupby('Borough').agg(np.std)['priceXmile']

In [21]:
import scipy
def tTest(mode1,mode2):
    one_sample_result  = scipy.stats.ttest_ind(mode1,mode2)
    return one_sample_result

In [23]:
k = datazone.Borough.value_counts()
t_test_result = []
list_borough = list(k.keys())
for i in list_borough:
    for j in list_borough:
        if i  !=j:
            result = tTest(datazone[datazone['Borough']==i]['priceXmile'],datazone[datazone['Borough']==j]['priceXmile'])
            t_test_result.append({i+"->"+j:result})

t_test_result

[{'Manhattan->Queens': Ttest_indResult(statistic=-0.02158979709322811, pvalue=0.9827751728861396)},
 {'Manhattan->Unknown': Ttest_indResult(statistic=-99.73581441240333, pvalue=0.0)},
 {'Manhattan->Brooklyn': Ttest_indResult(statistic=1.207191555156119, pvalue=0.22735841746847013)},
 {'Manhattan->Bronx': Ttest_indResult(statistic=-4.308991822135121, pvalue=1.640023995187033e-05)},
 {'Manhattan->EWR': Ttest_indResult(statistic=-185.58228259917897, pvalue=0.0)},
 {'Manhattan->Staten Island': Ttest_indResult(statistic=-1.7138731185545457, pvalue=0.08655207002847912)},
 {'Queens->Manhattan': Ttest_indResult(statistic=0.02158979709322811, pvalue=0.9827751728861396)},
 {'Queens->Unknown': Ttest_indResult(statistic=-31.489601456915956, pvalue=1.74654810237992e-217)},
 {'Queens->Brooklyn': Ttest_indResult(statistic=0.5023041849755552, pvalue=0.6154537461226044)},
 {'Queens->Bronx': Ttest_indResult(statistic=-1.8392377287035773, pvalue=0.0658807954969867)},
 {'Queens->EWR': Ttest_indResult(stat

***CRQ1 Part 2 : Price per mile wighted***

In [24]:
priceXmileWeighted= datazone['priceXmile']/datazone['minutes_trip_duration']

In [25]:
datazone.insert(loc=2, column='priceXmileWeighted', value= priceXmileWeighted)
datazone.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,priceXmileWeighted,priceXmile,trip_duration,trip_distance,PULocationID,DOLocationID,fare_amount,LocationID,Borough,minutes_trip_duration
0,2018-01-01 00:21:05,2018-01-01 00:24:23,2.727273,9.000000,00:03:18,0.5,41,24,4.5,41,Manhattan,3.30
1,2018-01-01 00:44:55,2018-01-01 01:03:05,0.285371,5.185185,00:18:10,2.7,239,140,14.0,239,Manhattan,18.17
2,2018-01-01 00:08:26,2018-01-01 00:14:21,1.266892,7.500000,00:05:55,0.8,262,141,6.0,262,Manhattan,5.92
3,2018-01-01 00:20:22,2018-01-01 00:52:51,0.101118,3.284314,00:32:29,10.2,140,257,33.5,140,Manhattan,32.48
4,2018-01-01 00:09:18,2018-01-01 00:27:06,0.280899,5.000000,00:17:48,2.5,246,239,12.5,246,Manhattan,17.80


In [28]:
#Run the mean of the new variable for each borough
means=datazone.groupby('Borough').agg(np.mean)['priceXmileWeighted']

In [29]:
#Run the standard deviation of the new variable for each borough
sds=datazone.groupby('Borough').agg(np.std)['priceXmile']

In [30]:
k = datazone.Borough.value_counts()
t_test_result = []
list_borough = list(k.keys())
for i in list_borough:
    for j in list_borough:
        if i  !=j:
            result = tTest(datazone[datazone['Borough']==i]['priceXmileWeighted'],datazone[datazone['Borough']==j]['priceXmileWeighted'])
            t_test_result.append({i+"->"+j:result})

t_test_result

C:\Users\Enzopc\Anaconda3\lib\site-packages\numpy\core\_methods.py:112: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Enzopc\Anaconda3\lib\site-packages\scipy\stats\stats.py:3842: RuntimeWarning: invalid value encountered in double_scalars
  d = mean1 - mean2
C:\Users\Enzopc\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\Enzopc\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\Enzopc\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


[{'Manhattan->Queens': Ttest_indResult(statistic=nan, pvalue=nan)},
 {'Manhattan->Unknown': Ttest_indResult(statistic=nan, pvalue=nan)},
 {'Manhattan->Brooklyn': Ttest_indResult(statistic=nan, pvalue=nan)},
 {'Manhattan->Bronx': Ttest_indResult(statistic=nan, pvalue=nan)},
 {'Manhattan->EWR': Ttest_indResult(statistic=nan, pvalue=nan)},
 {'Manhattan->Staten Island': Ttest_indResult(statistic=nan, pvalue=nan)},
 {'Queens->Manhattan': Ttest_indResult(statistic=nan, pvalue=nan)},
 {'Queens->Unknown': Ttest_indResult(statistic=nan, pvalue=nan)},
 {'Queens->Brooklyn': Ttest_indResult(statistic=nan, pvalue=nan)},
 {'Queens->Bronx': Ttest_indResult(statistic=nan, pvalue=nan)},
 {'Queens->EWR': Ttest_indResult(statistic=nan, pvalue=nan)},
 {'Queens->Staten Island': Ttest_indResult(statistic=nan, pvalue=nan)},
 {'Unknown->Manhattan': Ttest_indResult(statistic=nan, pvalue=nan)},
 {'Unknown->Queens': Ttest_indResult(statistic=nan, pvalue=nan)},
 {'Unknown->Brooklyn': Ttest_indResult(statistic=nan

In [ ]:
#?????????

plt.hist(datazone['priceXmile'], color = 'blue', edgecolor = 'black',
         bins = int(180/5))

# seaborn histogram
sns.distplot(datazone['priceXmile'], hist=True, kde=False, 
             bins=int(180/5), color = 'blue',
             hist_kws={'edgecolor':'black'})
# Add labels
plt.title('Histogram of priceXmile')

#or
# Density Plot and Histogram 
sns.distplot(datazone['priceXmile'], hist=True, kde=True, 
             bins=100, color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})